In [1]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

## Download and extract the datasets

* The datasets are located on Kaggle @ [www.kaggle.com/c/dogs-vs-cats/data](https://www.kaggle.com/c/dogs-vs-cats/data).
* Download and extract the dataset to `./data/` folder.

In [2]:
def create_small_dataset(target_dir='./data/dogs-vs-cats_small'):
    """Creates a smalled Cat/Dog dataset from the original Kaggle one."""
    import os, shutil
    original_dataset_dir = './data/dogs-vs-cats/train'
    
    def join_and_create(base, append):
        """Joins the `base` and `append` strings to create a folder path, and creates it."""
        directory = os.path.join(base, append)
        try:
            os.mkdir(directory)
        except OSError as e:
            print("[E] Cannot create {}, skipping...".format(directory))
        return directory
    def copy_files(f_pattern, f_range, destination):
        """Copies files from the original dataset directory. f_pattern must have 1 (one) parameter for `format`"""
        # f_names = [f_pattern.format(i) for i in range(f_range[0], f_range[1])]
        for fname in (f_pattern.format(i) for i in range(f_range[0], f_range[1])):
            src = os.path.join(original_dataset_dir, fname)
            dst = os.path.join(destination, fname)
            shutil.copyfile(src, dst)
    
    base_dir = join_and_create(target_dir, '')
    
    train_dir = join_and_create(base_dir, 'train')
    validation_dir = join_and_create(base_dir, 'validation')
    test_dir = join_and_create(base_dir, 'test')
    
    train_cats_dir = join_and_create(train_dir, 'cats')
    train_dogs_dir = join_and_create(train_dir, 'dogs')
    validation_cats_dir = join_and_create(validation_dir, 'cats')
    validation_dogs_dir = join_and_create(validation_dir, 'dogs')
    test_cats_dir = join_and_create(test_dir, 'cats')
    test_dogs_dir = join_and_create(test_dir, 'dogs')
    
    copy_files('cat.{}.jpg', (0, 1000), train_cats_dir)
    copy_files('cat.{}.jpg', (1000, 1500), validation_cats_dir)
    copy_files('cat.{}.jpg', (1500, 2000), test_cats_dir)
    copy_files('dog.{}.jpg', (0, 1000), train_dogs_dir)
    copy_files('dog.{}.jpg', (1000, 1500), validation_dogs_dir)
    copy_files('dog.{}.jpg', (1500, 2000), test_dogs_dir)
    
    return base_dir
    
base_dir = create_small_dataset()

[E] Cannot create ./data/dogs-vs-cats_small/, skipping...
[E] Cannot create ./data/dogs-vs-cats_small/train, skipping...
[E] Cannot create ./data/dogs-vs-cats_small/validation, skipping...
[E] Cannot create ./data/dogs-vs-cats_small/test, skipping...
[E] Cannot create ./data/dogs-vs-cats_small/train/cats, skipping...
[E] Cannot create ./data/dogs-vs-cats_small/train/dogs, skipping...
[E] Cannot create ./data/dogs-vs-cats_small/validation/cats, skipping...
[E] Cannot create ./data/dogs-vs-cats_small/validation/dogs, skipping...
[E] Cannot create ./data/dogs-vs-cats_small/test/cats, skipping...
[E] Cannot create ./data/dogs-vs-cats_small/test/dogs, skipping...


In [3]:
def make_model():
    from keras import layers
    from keras import models
    from keras import optimizers
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])
    return model

model = make_model()

Using TensorFlow backend.
